In [55]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import scale, StandardScaler
from sklearn.datasets import load_boston
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score,precision_recall_curve,confusion_matrix
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
classification = pd.read_csv('classification.csv')

classification

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1
5,1,1
6,1,1
7,0,1
8,0,0
9,0,1


In [84]:
TP = len(classification[(classification['true'] == 1) & (classification['pred'] == 1)].true)

TP

43

In [85]:
FP = len(classification[(classification['true'] == 0) & (classification['pred'] == 1)].true)

FP

34

In [86]:
FN = len(classification[(classification['true'] == 1) & (classification['pred'] == 0)].true)

FN

59

In [87]:
TN = len(classification[(classification['true'] == 0) & (classification['pred'] == 0)].true)

TN

64

In [90]:
TP, FP, FN, TN

(43, 34, 59, 64)

In [61]:
confusion_matrix(classification['true'], classification['pred'], labels=[1,0])

array([[43, 59],
       [34, 64]], dtype=int64)

In [23]:
round(accuracy_score(classification.true, classification.pred), 2)

0.54

In [20]:
round(precision_score(classification.true, classification.pred), 2)

0.56

In [21]:
round(recall_score(classification.true, classification.pred), 2)

0.42

In [22]:
round(f1_score(classification.true, classification.pred), 2)

0.48

In [24]:
scores = pd.read_csv('scores.csv')

scores

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263
5,0,0.108506,-0.790864,0.000000,0.000000
6,0,0.413749,-0.185143,0.283053,0.363636
7,0,0.961081,0.830973,1.000000,1.000000
8,0,0.055733,-0.872805,0.000000,0.100000
9,0,0.168708,-0.518390,0.000000,0.100000


In [40]:
roc_auc = 0
n = 0
for i in range(len(scores.columns[1:])):
    rc = roc_auc_score(scores.true, scores[scores.columns[1:][i]])
    
    if rc > roc_auc:
        roc_auc = rc
        n = i

In [83]:
scores.columns[1:][n], roc_auc

('score_tree', 0.719187675070028)

In [80]:
maxpr = 0
n = 0
for i in range(len(scores.columns[1:])):
    
    precision, recall, thresholds = precision_recall_curve(scores.true, scores[scores.columns[1:][i]])
    
    d = {'precision': precision,'recall': recall}
    
    pr = max(df[df['recall'] >= 0.7].precision)
    
    if rc > maxpr:
        maxpr = pr
        n = i

In [82]:
scores.columns[1:][n], maxpr

('score_tree', 0.6517857142857143)